In [1]:
print("Agarwal")

Agarwal


In [2]:
%pwd

'c:\\Users\\rames\\Medical-Chatbox\\research'

In [8]:
import os
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\rames\\Medical-Chatbox'

In [42]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [43]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents=loader.load();
    
    return documents

In [44]:
extracted_data = load_pdf_file(data='Data/')

incorrect startxref pointer(1)
parsing for Object Streams


In [45]:
extracted_data

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '200

In [46]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [47]:
text_chunks = text_split(extracted_data)
print("Length of the text chunks", len(text_chunks))

Length of the text chunks 0


In [ ]:
#text_chunks

NameError: name 'text_chunks' is not defined

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))


In [ ]:
#query_result

NameError: name 'download_hugging_face_embeddings' is not defined

In [ ]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [20]:

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

TypeError: str expected, not NoneType

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embeddings=embeddings,
)

In [ ]:
#load existing index
from langchain_pinecone import PineConeVectorStore

docsearch = PineConeVectorStore.existing_index(
    index_name=index_name,
    embeddings=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("what is Acne?")

In [ ]:
retrieved_docs

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature =0.4, max_tokens=500)

ImportError: cannot import name '_set_tracing_context' from 'langsmith.run_helpers' (c:\Users\rames\anaconda3\envs\medibot\lib\site-packages\langsmith\run_helpers.py)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_document_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "you are an assistant for question-answering tasks."
    "Use the following peices of retrieved context to answer"
    "the question.if you dont know the answer , say that you"
    "dont know.use three sentences maximum and keep the "
    "answer consise "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human", "{input}"),
    ]
)


ImportError: cannot import name 'create_retrieval_chain' from 'langchain.chains' (c:\Users\rames\anaconda3\envs\medibot\lib\site-packages\langchain\chains\__init__.py)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])